In [1]:
import findspark
findspark.init('/home/srabin/spark-3.5.1-bin-hadoop3')
import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('my_tree').getOrCreate()

24/08/07 16:26:51 WARN Utils: Your hostname, DESKTOP-P0KTK6U resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/07 16:26:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/07 16:26:51 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/07 16:26:52 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/08/07 16:26:52 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.
24/08/07 16:26:52 WARN Utils: Service 'SparkUI' could not bind on port 4042. Attempting port 4043.
24/08/07 16:26:52 WARN Utils: Service 'SparkUI' could not bind on port 4043. Attempting port 4044.
24/08/07 16:26:52 WARN Utils: Service 'SparkUI' could not bind on port 4044. Attempting port

In [3]:
from pyspark.ml import Pipeline

In [4]:
from pyspark.ml.classification import RandomForestClassifier, GBTClassifier, DecisionTreeClassifier

In [5]:
#from pyspark.ml.regression import RandomForestRegressor, GBTRegressor, DecisionTreeRegressor

In [6]:
data =  spark.read.format('libsvm').load('sample_libsvm_data.txt')

24/08/07 16:31:31 WARN LibSVMFileFormat: 'numFeatures' option not specified, determining the number of features by going though the input. If you know the number in advance, please specify it via 'numFeatures' option to avoid the extra scan.


In [7]:
data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [8]:
train_data, test_data = data.randomSplit([0.7, 0.3])

In [10]:
dtc = DecisionTreeClassifier()
rfc = RandomForestClassifier(numTrees = 100)
gbt = GBTClassifier()

In [11]:
dtc_model = dtc.fit(train_data)
rfc_model = rfc.fit(train_data)
gbt_model = gbt.fit(train_data)

In [13]:
dtc_preds = dtc_model.transform(test_data)
rfc_preds = rfc_model.transform(test_data)
gbt_preds = gbt_model.transform(test_data)

In [14]:
dtc_preds.show()

+-----+--------------------+-------------+-----------+----------+
|label|            features|rawPrediction|probability|prediction|
+-----+--------------------+-------------+-----------+----------+
|  0.0|(692,[98,99,100,1...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[122,123,124...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[128,129,130...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[152,153,154...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  0.0|(692,[154,155,156...|   [0.0,37.0]|  [0.0,1.0]|       1.0|
|  0.0|(692,[154,155,156...|   [0.0,37.0]|  [0.0,1.0]|       1.0|
|  0.0|(692,[181,182,183...|   [33.0,0.0]|  [1.0,0.0]|       0.0|
|  1.0|(692,[100,101,102...|   [0.0,37.0]|  [0.0,1.0]|       1.0|
|  1.0|(692,[119,120,121...|   [0.0,37.0]|  [0.0,1.0]|       1.0|
|  1.0|(69

In [15]:
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

In [16]:
acc_eval = MulticlassClassificationEvaluator(metricName = 'accuracy')

In [17]:
print('Decision Tree Clasifier Accuracy:')
acc_eval.evaluate(dtc_preds)

Decision Tree Clasifier Accuracy:


0.9333333333333333

In [18]:
print('Random Forest Clasifier Accuracy:')
acc_eval.evaluate(rfc_preds)

Random Forest Clasifier Accuracy:


1.0

In [20]:
print('Gradiant Bosster Clasifier Accuracy:')
acc_eval.evaluate(gbt_preds)

Gradiant Bosster Clasifier Accuracy:


24/08/07 16:49:09 WARN InstanceBuilder: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS


0.9333333333333333

In [21]:
rfc_model.featureImportances

SparseVector(692, {99: 0.0006, 102: 0.0006, 131: 0.0003, 155: 0.0003, 206: 0.0016, 207: 0.0066, 214: 0.002, 232: 0.0019, 234: 0.0022, 235: 0.0005, 239: 0.0007, 241: 0.0004, 258: 0.0006, 263: 0.0009, 268: 0.0006, 271: 0.0008, 272: 0.0006, 273: 0.0159, 274: 0.0046, 288: 0.0007, 290: 0.0138, 293: 0.0005, 295: 0.0008, 299: 0.0005, 300: 0.007, 302: 0.0016, 316: 0.005, 317: 0.0015, 318: 0.0034, 319: 0.0009, 323: 0.0077, 324: 0.0067, 327: 0.0014, 344: 0.0092, 345: 0.0077, 346: 0.0019, 347: 0.0031, 350: 0.0041, 351: 0.028, 352: 0.0061, 356: 0.0093, 357: 0.0139, 358: 0.0005, 370: 0.0006, 372: 0.0021, 373: 0.0033, 374: 0.0027, 375: 0.0005, 377: 0.004, 378: 0.0238, 379: 0.0374, 380: 0.0069, 381: 0.0005, 385: 0.0068, 386: 0.0071, 387: 0.0007, 400: 0.0128, 405: 0.0376, 406: 0.0383, 407: 0.0413, 408: 0.0069, 409: 0.0009, 410: 0.0026, 411: 0.0034, 412: 0.0069, 415: 0.0007, 427: 0.0063, 430: 0.0007, 433: 0.0403, 434: 0.087, 435: 0.0145, 436: 0.0011, 442: 0.0069, 453: 0.0015, 455: 0.016, 456: 0.0067, 4